In [1]:
import pickle

with open('../../harvard/repos/dcv-python/32.pickle', 'rb') as handle:
    dcvs = pickle.load(handle)

In [2]:
_ids = list(dcvs.keys())

In [3]:
from fastbook import tensor, show_image

/Users/yazatian/Library/Caches/pypoetry/virtualenvs/dcv-python-ytIwIEW1-py3.9/lib/python3.9/site-packages/fastbook/__init__.py:19: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [4]:
import numpy as np

In [5]:
with open('../../harvard/repos/dcv-python/dcvs.pickle', 'rb') as handle:
    real_image = pickle.load(handle)

In [6]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

In [7]:
circular_mask = create_circular_mask(40,40, radius=12.5)

In [8]:
final_cutouts = {}
for x in _ids:
    final_cutouts[x] = np.where(circular_mask==1, real_image[x], circular_mask)[4:36,4:36]

In [9]:
from feature_embedding import *

In [10]:
train_dataset = []
labels = []
for key, value in dcvs.items():
    train_dataset.append(tensor(np.expand_dims(np.float32(value), axis=0)))
    labels.append(key)

In [11]:
img_to_id = {}
for i, x in enumerate(train_dataset):
    img_to_id[np.array(x).tobytes()] = labels[i]

In [12]:
import pytorch_lightning as pl

In [13]:
labels[0]

2359120

In [14]:
# Setting the seed
pl.seed_everything(42)

Global seed set to 42


42

In [15]:
train_dataset_torch = torch.stack(train_dataset) / 255

In [16]:
train_dataset_torch.shape

torch.Size([42162, 1, 32, 32])

In [17]:
train_dataset_torch.max()

tensor(1.)

In [18]:
train_set, val_set, test_set = torch.utils.data.random_split(train_dataset_torch, 
                                                             [33730, 4216, 4216], 
                                                             generator=torch.Generator().manual_seed(42))

In [19]:
len(train_set)

33730

In [20]:
#Hyperparameters
training_steps = 5000
train_batch_size = 8
holdout_batch_size = 1
loss_fn = torch.nn.MSELoss()

In [21]:
# We define a set of data loaders that we can use for various purposes later.
#pin_memory=True
train_loader = DataLoader(train_set, batch_size=train_batch_size)
val_loader = DataLoader(val_set, batch_size=holdout_batch_size)
test_loader = DataLoader(test_set, batch_size=holdout_batch_size)

In [22]:
train_set[0].shape[-1:None:-1]

torch.Size([32, 32, 1])

In [23]:
model = AE(input_size=train_set[0].shape[-1:None:-1],depth=3)

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [25]:
from torchsummary import summary
import datetime
from torch.utils.tensorboard import SummaryWriter

In [26]:
import os

In [27]:
summary(model,input_size=train_set[0].shape)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
writer = SummaryWriter(logdir)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]             640
    InstanceNorm2d-2           [-1, 64, 32, 32]               0
              ReLU-3           [-1, 64, 32, 32]               0
              ReLU-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,928
    InstanceNorm2d-6           [-1, 64, 32, 32]               0
              ReLU-7           [-1, 64, 32, 32]               0
              ReLU-8           [-1, 64, 32, 32]               0
         MaxPool2d-9           [-1, 64, 16, 16]               0
           Conv2d-10          [-1, 128, 16, 16]          73,856
   InstanceNorm2d-11          [-1, 128, 16, 16]               0
             ReLU-12          [-1, 128, 16, 16]               0
             ReLU-13          [-1, 128, 16, 16]               0
           Conv2d-14          [-1, 128,

In [28]:
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
train_dataset[0].shape

torch.Size([1, 32, 32])

In [30]:
#x = torch.unsqueeze(train_dataset[0],0)

In [ ]:
#cosine embedding loss 

In [31]:
train(train_loader, val_loader, model, loss_fn, 
      optimizer, training_steps, 
      device=device,writer=writer)

  0%|          | 0/5000 [00:00<?, ?it/s]

0.005212962
0.0033876116
0.0029022251
0.0026324524
0.0024601428
0.002347675
0.0022566274
0.0021625764
0.002122362
0.002055774
0.002000078
0.0019413242
0.0019012543
0.001867778
0.0018225302
0.0017830794
0.0017688064
0.0017331024


KeyboardInterrupt: 

In [ ]:
model.eval()

writer.close()

if not(os.path.isdir("models")):
    os.mkdir("models")

model_name = f"{os.path.basename(logdir)}.pth"
model_name_fullpath = os.path.join("models",model_name)
torch.save(model,model_name_fullpath)

print(f"Logged in: {logdir}")
print(f"Model saved to: {model_name_fullpath}")